In [269]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import geocoder
import requests
#!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

#! pip install folium
import folium

#libraries for folium dots
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans 

#### 1. Get data of Toronto boroughs

In [237]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html5lib')
tables = soup.find_all('table')
toronto_bor_soup = tables[0]

In [238]:
#Create DataFrame and put wiki-data into it

tor_data = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood'])


row_ex = toronto_bor_soup.tbody.find_all("tr")[0]
col_ex = row_ex.find_all("td")[3]

for row in toronto_bor_soup.tbody.find_all("tr"):
    for col in row.find_all("td"):
        if (not ('ot assign' in col.text)):    
            find_M = col.text.find("M")
            find_kb = col.text.find("(")
            find_ke = col.text.find(")")

            bor_code = col.text[find_M:(find_M+3)]
            bor_name = col.text[(find_M+3):(find_kb)]
            bor_neigh = col.text[(find_kb+1):(find_ke)]
            bor_neigh = bor_neigh.replace(" /", ",")
            #print(' Code: {} \n Name: {} \n Neighbour: {}'.format(bor_code, bor_name, bor_neigh))
            tor_data = tor_data.append({"PostalCode" : bor_code, "Borough" : bor_name, "Neighbourhood" : bor_neigh}, ignore_index=True)

tor_data['Borough']=tor_data['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
            
tor_data.head()
#print('Number of rows in the Toronto Borough data frame are', tor_data.shape[0])

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### Set up Foursquare  / geolocator 

In [192]:
'''

geolocator = Nominatim(user_agent="CourseraIBM_DataSience_MM")

#address = tor_data['PostalCode'][0], ', Toronto'

tor_data['lat'] = ""
tor_data['lon'] = ""

for index, row in tor_data.iterrows():
    
    #address = row['PostalCode'] + ', ' + row['Neighbourhood'] + ', Toronto'
    address = row['PostalCode'] + ', Toronto'
    print(address)
    
    location = geolocator.geocode("address")
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
  
        tor_data['lat'][tor_data['PostalCode'] == row['PostalCode']] = latitude
        tor_data['lon'][tor_data['PostalCode'] == row['PostalCode']] = longitude

#tor_data.head()
'''

'\n\ngeolocator = Nominatim(user_agent="CourseraIBM_DataSience_MM")\n\n#address = tor_data[\'PostalCode\'][0], \', Toronto\'\n\ntor_data[\'lat\'] = ""\ntor_data[\'lon\'] = ""\n\nfor index, row in tor_data.iterrows():\n    \n    #address = row[\'PostalCode\'] + \', \' + row[\'Neighbourhood\'] + \', Toronto\'\n    address = row[\'PostalCode\'] + \', Toronto\'\n    print(address)\n    \n    location = geolocator.geocode("address")\n    if location is not None:\n        latitude = location.latitude\n        longitude = location.longitude\n  \n        tor_data[\'lat\'][tor_data[\'PostalCode\'] == row[\'PostalCode\']] = latitude\n        tor_data[\'lon\'][tor_data[\'PostalCode\'] == row[\'PostalCode\']] = longitude\n\n#tor_data.head()\n'

### 3. Retrieve Longitude/Latitude from .csv file and join with PostalCodes

In [239]:
coord_data = pd.read_csv('C:/Users/Mario/Desktop/Sortieren/3_Unterlagen/11_IBM_data science/9_capstone_project/geospatial_coordinates.csv')
coord_data.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
tor_data = tor_data.merge(coord_data, on='PostalCode', how='left')
tor_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### 4. Create a Toronto map

In [247]:
# create map
tor_center_lat = tor_data['Latitude'][74:75]
tor_center_lon = tor_data['Longitude'][74:75]
map_toronto = folium.Map(location=[tor_center_lat, tor_center_lon], zoom_start=11)

for lat, lng, borough, neighborhood in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['Borough'], tor_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
       
map_toronto

### 5. Do K-Means cluster based on distances

In [275]:
n_clusters = 10
k_means = KMeans(init="k-means++", n_clusters= n_clusters, n_init=12)

X = tor_data[['Longitude', 'Latitude']]
k_means.fit(X)

k_means_labels = k_means.labels_
k_means_clusters = k_means.cluster_centers_

tor_data['Cluster_n'] = k_means_labels

### 6. Do Cluster-Plot Toronto

In [276]:
# create map
tor_center_lat = tor_data['Latitude'][74:75]
tor_center_lon = tor_data['Longitude'][74:75]
map_toronto = folium.Map(location=[tor_center_lat, tor_center_lon], zoom_start=11)

#set color for clusters
x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


for lat, lng, borough, neighborhood, cluster in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['Borough'], tor_data['Neighbourhood'], tor_data['Cluster_n']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
       
map_toronto

The cluster is based on the coordinates. This can be more precised by other criteria like before New York, or inhabitants etc.